### 여러 페이지에서 가져온 배우 정보를 MongoDB 에 저장하기

In [ ]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

In [ ]:
'''
MongoDB 연결하기
'''

conn = pymongo.MongoClient()
cine = conn.cine
collection_cine2 = cine.collection_cine2


In [ ]:
'''
collection 삭제
'''
# collection_cine.drop()

In [ ]:
actors_info_list = []

In [ ]:
url = "http://www.cine21.com/rank/person/content"

post_data = {}
post_data["section"] = "actor"
post_data["period_start"] = "2020-12"
post_data["gender"] = "all"


In [ ]:
# 1 page ~ 20 page


for idx in range(1, 21):

    post_data["page"] = idx

    res = requests.post(url, data=post_data)
    soup = BeautifulSoup(res.content, "html.parser")


    actors = soup.select("li.people_li div.name")
    hits   = soup.select("ul.num_info > li > strong")
    movies = soup.select("ul.mov_list")
    rankings = soup.select("li.people_li > span.grade")

    for idx, actor in enumerate(actors):
        actor_name   = (actor.text).split("(")[0]
        actor_hits   = int(hits[idx].text.replace(",",""))
        movie_titles = movies[idx].select("li a span")

        movie_title_list = []
        for movie_title in movie_titles:            
            movie_title_list.append(movie_title.text)

        actors_info_dict = { }
        actors_info_dict["배우이름"] = actor_name
        actors_info_dict["흥행지수"] = actor_hits
        actors_info_dict["출연영화"] = movie_title_list
        actors_info_dict["랭킹"] = rankings[idx].text


        actor_link = "http://www.cine21.com" + actor.select_one("a").attrs["href"]
        actor_resp = requests.get(actor_link)    

        actor_soup = BeautifulSoup(actor_resp.content, "html.parser")

        default_info  = actor_soup.select_one("ul.default_info")
        actor_details = default_info.select("li")


        for actor_item in actor_details:           
            actor_item_key = actor_item.select_one("span.tit").text
            actor_item_value = re.sub("<span.*?>.*?</span>", "", str(actor_item))
            actor_item_value = re.sub("<.*?>", "", actor_item_value)

            actors_info_dict[actor_item_key] = actor_item_value

        actors_info_list.append(actors_info_dict)    

    
actors_info_list  


In [ ]:
# collection_cine.delete_many({})

In [ ]:
collection_cine2.insert_many(actors_info_list)

In [ ]:
collection_cine2.count_documents({})

### 데이터 분석 준비하기
#### 1) key(field)명 변경하기
- MongoDB 에 저장되어 있는 document 의 key(field) 이름 변경하기
- update_one() / update_many()

In [ ]:
collection_cine2.find_one({})

In [ ]:
import pprint

documents = collection_cine2.find({}).limit(3)

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

In [ ]:
'''
배우이름 --> 배우 이름  : $rename
'''
collection_cine2.update_many({}, {"$rename" : {"배우이름":"배우 이름"}})

In [ ]:
documents = collection_cine2.find({}).limit(3)

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

### 조회할 때 정렬하기(오름차순 / 내림차순)
- sort() 메소드
- 오름차순 : pymongo.ASCENDING (기본값)
- 내림차순 : pymongo.DESCENDING

In [ ]:
documents = collection_cine2.find({}).sort("생년월일").limit(10)

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

In [ ]:
documents = collection_cine2.find({}).sort("생년월일", pymongo.ASCENDING).limit(10)

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

In [ ]:
documents = collection_cine2.find({}).sort("생년월일", pymongo.DESCENDING).limit(10)

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

In [ ]:
'''
"특기"라는 key 값이 있는 document 만 조회하기
                  ㄴ $exists
'''
documents = collection_cine2.find({"특기" : {"$exists" : True}})

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

In [ ]:
'''
"특기" 라는 key 값이 있는 배우들만 
"흥행지수" 순으로 정렬하기 (오름차순)
'''
documents = collection_cine2.find({"특기" : {"$exists" : True}}).sort("흥행지수")

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

In [ ]:
'''
"특기" 라는 key 값이 있는 배우들만 
"흥행지수" 내림차순으로 정렬하기 - 10 개만
'''
documents = \
  collection_cine2.find({"특기" : {"$exists" : True}})\
                 .sort("흥행지수", pymongo.DESCENDING)\
                 .limit(10)

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

In [ ]:
'''
생년월일 이라는 key 값이 있는 배우들만
흥행지수 내림차순으로 정렬하기 - 5 개만
'''
documents = \
  collection_cine2.find({"생년월일" : {"$exists" : True}})\
                 .sort("흥행지수", pymongo.DESCENDING)\
                 .limit(5)

for document in documents:
    pprint.pprint(document)
    print("-" * 80)

In [ ]:
'''
생년월일 key 값이 있는 배우들만
흥행지수 순서로 정렬해서 출력하기
내림차순, 5 명만 
배우 이름 key 만 출력하세요
'''

documents = \
  collection_cine2.find({"생년월일" : {"$exists":True}}, {"배우 이름":1, "흥행지수":1,"_id":0})\
                 .sort("흥행지수").limit(10)

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

In [ ]:
'''
내림차순 : pymongo.DESCENDING
'''
documents = \
  collection_cine2.find({"생년월일" : {"$exists":True}}, {"배우 이름":1, "흥행지수":1,"_id":0})\
                 .sort("흥행지수", pymongo.DESCENDING).limit(10)

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

In [ ]:
'''
흥행지수가 10000 이상인 배우만 조회하기
'''
documents = collection_cine2.find({"흥행지수":{"$gte":10000}})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

In [ ]:
'''
흥행지수가 10000 이상인 배우만 조회하기
흥행지수, 배우 이름만 출력하기
흥행지수 내림차순으로 정렬하기
'''
documents = \
  collection_cine2.find({"흥행지수":{"$gte":5000}}, \
                       {"흥행지수":1,"배우 이름":1, "출연영화":1,"_id":0})\
                 .sort("흥행지수", pymongo.DESCENDING)

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

In [ ]:
'''
흥행지수가 10000 이상인 배우 중에서
"모가디슈" 라는 영화에 출연한 사람만 조회하기
배우 이름, 흥행지수, 출연영화 만 조회(출력)하기
'''
documents = \
  collection_cine2.find({"흥행지수":{"$gte":10000}, "출연영화":"비상선언"},\
                       {"배우 이름":1, "흥행지수":1, "출연영화":1, "_id":0})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

In [ ]:
'''
흥행지수가 5000 이 넘으면서
"비상선언" 또는 "이웃사촌" 에 출연한 배우들을 조회하세요
배우 이름, 출연영화 키만 출력하세요
'''
documents = \
  collection_cine2.find({"흥행지수":{"$gte":5000}, "출연영화":"비상선언"},\
                       {"배우 이름":1, "출연영화":1, "_id":0})

documents = \
  collection_cine2.find({"흥행지수":{"$gte":5000}, "출연영화":"이웃사촌"},\
                       {"배우 이름":1, "출연영화":1, "_id":0})

### find() - or

In [ ]:
documents = \
  collection_cine2.find({"흥행지수":{"$gte":5000}, \
                        "$or" : [{"출연영화":"비상선언"}, {"출연영화":"이웃사촌"}]},\
                       {"배우 이름":1, "출연영화":1, "흥행지수":1,"_id":0})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

### find() - and

In [ ]:
documents = \
  collection_cine2.find({"흥행지수":{"$gte":5000}, \
                        "$and" : [{"출연영화":"비상선언"}, {"출연영화":"그것만이 내 세상"}]},\
                       {"배우 이름":1, "출연영화":1, "흥행지수":1,"_id":0})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

### find() - nor (not or)

In [ ]:
documents = \
  collection_cine2.find({"$nor":[{"흥행지수": {"$gte":10000}}, {"흥행지수":{"$lte":1000}}]}, \
                       {"배우 이름":1, "흥행지수":1, "_id":0})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

In [ ]:
documents = \
  collection_cine2.find({"$and":[{"흥행지수": {"$lte":10000}}, {"흥행지수":{"$gte":1000}}]}, \
                       {"배우 이름":1, "흥행지수":1, "_id":0})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

In [ ]:
documents = \
  collection_cine2.find({"$or":[{"흥행지수": {"$gte":10000}}, {"흥행지수":{"$lte":1000}}]}, \
                       {"배우 이름":1, "흥행지수":1, "_id":0})

for document in documents:
    pprint.pprint(document)
    print('-' * 80)

### find() - in(포함되어 있음), nin(not in)(포함되어 있지 않음)

In [ ]:
'''
흥행지수가 12619, 15263 인 배우만 조회하기 - $in 
배우 이름, 흥행지수 만 조회하기
'''
documents = collection_cine2.find({"흥행지수":{"$in":[12619, 15263]}},\
                                 {"배우 이름":1, "흥행지수":1, "_id":0})
                                  
for document in documents:
    pprint.pprint(document)    

In [ ]:
'''
흥행지수가 12619, 15263 이 아닌 배우만 조회하기 -  $nin
배우 이름, 흥행지수만 조회하기
'''
documents = collection_cine2.find({"흥행지수":{"$nin":[12619, 15263]}},\
                                 {"배우 이름":1, "흥행지수":1, "_id":0})
for document in documents:
    pprint.pprint(document)    

In [ ]:
'''
흥행지수가 12619, 15263 이 아니고(아니면서)
10000 이상인 배우 이름과 흥행지수를 조회하세요
  ㄴ 10000 미만이 아닌
[힌트]
  $nor 와 $in 같이 사용하기
  
1)   흥행지수가 12619, 15263 인 조건

     {"흥행지수":{"$in":[12619, 15263]}
     
     
2)   흥행지수가 10000 미만인 조건

     {"흥행지수":{"$lt":10000}
  
3)   {"$nor" : [{"흥행지수":{"$in":[12619, 15263]}, {"흥행지수":{"$lt":10000}]}
'''
documents = \
  collection_cine2.find({"$nor" : [{"흥행지수":{"$in":[12619, 15263]}}, \
                                  {"흥행지수":{"$lt":10000}}]},\
                       {"배우 이름":1, "흥행지수":1, "_id":0})

for document in documents:
    pprint.pprint(document)
    

### find() - skip() / limit()
- skip(n) - 시작 부분에서 n 개 건너뜀 (n+1 부터 검색함)
- limit(n) - n 개만 조회함

In [ ]:
'''
흥행지수가 10000 을 초과하는 배우들의
배우 이름, 흥행지수를 조회하세요
'''
documents = \
  collection_cine2.find({"흥행지수":{"$gt":10000}},\
                       {"흥행지수":1, "배우 이름":1, "출연영화":1,"_id":0})

for document in documents:
    pprint.pprint(document)
print('-' * 80)    
  

In [ ]:
'''
모가디슈에 출연한 배우들의 배우 이름, 출연영화를 출력하세요
'''
documents = \
  collection_cine2.find({"출연영화":"모가디슈"},\
                       {"배우 이름":1, "출연영화":1, "_id":0})
for document in documents:
    pprint.pprint(document)

In [ ]:
'''
모가디슈나 타짜에 출연한 배우들의 
배우 이름, 출연영화를 조회하세요
'''
documents = \
  collection_cine2.find({"$or":[{"출연영화":"모가디슈"},{"출연영화":"타짜"}]},\
                        {"배우 이름":1, "출연영화":1, "_id":0})
for document in documents:
    pprint.pprint(document)
    

In [119]:
'''
모가디슈와 타짜에 출연한 배우들의 
배우 이름, 출연영화를 조회하세요
'''
documents =\
  collection_cine2.find({"출연영화":{"$all":["모가다슈", "타짜"]}},\
                       {"배우 이름":1, "출연영화":1, "_id":0})
for document in documents:
    pprint.pprint(document)

In [ ]:
documents = collection_cine2.find({},{"_id":0})
# print(len(list(documents)))
for document in documents:
    pprint.pprint(document)

In [ ]:
'''
5 편의 영화에 출연한 배우만 검색하기 : $size
배우 이름, 출연영화
'''

documents = \
  collection_cine2.find({"출연영화":{"$size":5}}, {"배우 이름":1,"출연영화":1,"_id":0})

for document in documents:
    pprint.pprint(document)
    

### elemMatch
- 각 document 의 key의 값으로 저장되어 있는 list에
- 적어도 한 개 이상의 요소(element : item)가 여러 개의 조건을 
- 동시에 만족하는 경우
- 조건인 하나인 경우에는 사용하지 않음

In [ ]:
documents = collection_cine2.find({}).limit(3)
for document in documents:
    pprint.pprint(document)

In [ ]:
'''
직업이 가수인 배우 중에서 
흥행지수가 가장 높은 순서로 (내림차순) 정렬해서
배우 이름, 직업, 흥행 지수를 조회하세요
'''
documents = collection_cine2.find({"직업":"가수"},\
                                 {"배우 이름":1, "직업":1, "흥행지수":1, "_id":0})\
                           .sort("흥행지수", pymongo.DESCENDING) 
for document in documents:
    pprint.pprint(document)

In [ ]:
'''
collection_elementmatch 컬렉션 생성하기
'''
cine.collection_elementmatch
# cine["collection_elementmatch"]

In [ ]:
elematch = cine.collection_elementmatch

In [ ]:
doc1 = {"results" : [81, 85, 89]}
doc2 = {"results" : [81, 85, 89, 72]}
doc3 = {"results" : [81, 85, 89, 97]}
doc4 = {"results" : [81, 85, 89, 72, 97]}
doc5 = {"results" : [77, 86, 93]}

elematch.insert_many(
    [
        doc1, doc2, doc3, doc4, doc5
    ]
)

In [ ]:
'''
"$gte":90  <- 90 점 이상인 data 가 있는 document 조회(검색)하기
"$lt":80   <- 80 점 미만인 data 가 있는 document 조회하기
'''
documents = elematch.find({"results":{"$gte":90, "$lt":80}}, {"_id":0})
for document in documents:
    pprint.pprint(document)

In [ ]:
'''
각 조건별로 하나라도 만족하는 element가 있는 경우
'''
documents = elematch.find({"results":{"$elemMatch":{"$gte":85, "$lt":90}}}, {"_id":0})
for document in documents:
    print(document)

In [ ]:
doc6 = {"results": [82, 85, 88]}
doc7 = {"results": [92, 95, 98]}

elematch.insert_many(
    [
        doc6, doc7        
    ]
)

In [ ]:
for document in elematch.find({"results" : {"$elemMatch":{"$gte":80, "$lt":85}}}, {"_id":0}):
    print(document)

In [ ]:
'''
collection_cine2 에서
출연영화 key 만 10 개 조회하기
'''
for document in collection_cine2.find({}, {"출연영화":1, "_id":0}).limit(10):
    print(document)

In [ ]:
'''
출연영화에 '도굴' 이라는 영화가 있는 document 만 조회하기
출연영화 key 만 출력함
'''
for document in collection_cine2.find({"출연영화":"도굴"}, {"출연영화":1, "_id":0}):
    print(document)

In [106]:
'''
출연영화에 '모가디슈' 이라는 영화가 포함되어 있는 document 만 조회하기
출연영화 key 만 출력함
'''
for document in collection_cine2.find({"출연영화":"모가디슈"}, {"출연영화":1, "_id":0}):
    print(document)

{'출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987']}
{'출연영화': ['모가디슈', '클래식']}
{'출연영화': ['모가디슈', '비상선언', '스프링 송', '아이 캔 스피크', '미성년']}
{'출연영화': ['모가디슈', '국가부도의 날']}
{'출연영화': ['모가디슈', '지푸라기라도 잡고 싶은 짐승들', '헌트', '똥파리', '군도 : 민란의 시대', '끝까지 간다']}
{'출연영화': ['싱크홀', '모가디슈', '괴기맨숀', '액션히어로', '싸나희 순정', '숏버스 이별행']}
{'출연영화': ['모가디슈', '메기', '우리 손자 베스트', '반도', '오늘영화']}
{'출연영화': ['모가디슈', '리미트', '메기', '1987']}


In [107]:
'''
출연영화가 "모가디슈", "클래식" 두 개인 document 조회하기
'''
for document in collection_cine2.find({"출연영화":["모가디슈", "클래식"]}, {"출연영화":1, "_id":0}):
    print(document)

{'출연영화': ['모가디슈', '클래식']}


In [108]:
'''
출연영화가 "모가디슈", "메기" 두 개인 document 조회하기
'''
for document in collection_cine2.find({"출연영화":["모가디슈", "메기"]}, {"출연영화":1, "_id":0}):
    print(document)

In [ ]:
'''
출연영화에 "모가디슈", "메기" 가 포함되어 있는 document
'''
documents = \
  collection_cine2.find({"출연영화":{"$all":["모가디슈", "메기"]}}, \
                        {"출연영화":1, "_id":0})

for document in documents:
    print(document)

In [ ]:
'''
출연영화에 "국가부도의 날" 이 포함되어 있는 document
'''
for document in collection_cine2.find({"출연영화":"국가부도의 날"}, {"출연영화":1,"_id":0}):
    print(document)


In [116]:
'''
"국가부도의 날" 에 출연한 배우들 중에서 
"흥행지수" 가 높은 순으로 
배우 이름, 직업, 흥행지수를 조회하세요
'''
documents = collection_cine2.find({"출연영화":"국가부도의 날"},\
                                  {"배우 이름":1, "직업":1, "흥행지수":1, "_id":0})\
                            .sort("흥행지수", pymongo.DESCENDING)
for document in documents:
    pprint.pprint(document)

{'배우 이름': '조우진', '직업': '배우', '흥행지수': 11621}
{'배우 이름': '허준호', '직업': '배우', '흥행지수': 9379}
{'배우 이름': '김홍파', '직업': '배우', '흥행지수': 4805}
